## OR-Tools

### Overview

When exploring Python packages for tackling mixed-integer programming (MIP) and various optimization challenges, it's crucial to distinguish between two key components: the front-end, where you define your problem (including the objective function and constraints), and the solver, which works behind the scenes to find a solution. Here are some of the more common options for solving MIP problems in Python:

* [milp in Scipy](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.milp.html#scipy.optimize.milp), acting as a wrapper for [HiGHS linear optimization software](https://highs.dev/),
* [Pyomo](http://www.pyomo.org/), which is also used by [Python for Power System Analysis (PyPSA)](https://pypsa.org/index.html),
* [PuLP](https://github.com/coin-or/pulp), another popular choice for defining optimization models.

However, a noteworthy contender in this field is [OR-Tools from Google](https://developers.google.com/optimization). According to the [OR-Tools webside](https://developers.google.com/optimization), OR-Tools is: 

>  [...] an open source software suite for optimization, tuned for tackling the world's toughest problems in vehicle routing, flows, integer and linear programming, and constraint programming.

As of February 10, 2024, the project had 10.2k stars on [GitHub](https://github.com/google/or-tools). For comparison, Pyomo had only 1.8k stars on [GitHub](https://github.com/Pyomo/pyomo).

What I like about OR-Tools is the fact that a tech giant like Google is standing behind it. Google obviously has a lot of optimization problems to solve and hence they have a vested interest in the continuous development and maintenance of this project. What I don't like is the API reference. The [website](https://developers.google.com/optimization) with guides and examples is okay. However, the Python API reference could use more love even though there exist two versions, namely one build with [pdoc](https://or-tools.github.io/docs/pdoc/ortools.html) and one [doxygen version](https://or-tools.github.io/docs/python/index.html).

It is worth mentioning that OR-Tools comes with an interface to several commercial and non-commercial solvers, offering flexibility in choosing the right tool for the job. In addition, you can also make use of Google's own solver for linear optimization problems, which is called [Glop](https://github.com/google/or-tools/tree/stable/ortools/glop).


### Installation

The OR-Tools package is available on [PyPI](https://pypi.org/project/ortools/) and can be installed using pip:
```
pip install ortools
```

For this blog post, I am using version 9.8.3296, which was the latest one I could install.


## The unit-commitment and dispatch model

Let's dive into solving a straightforward unit-commitment and dispatch problem using mixed-integer programming (MIP) with OR-Tools. We'll explore both the problem formulation and the coding aspect side by side, providing a clear, step-by-step guide to under how OR-Tools works.

### Solver
To begin, we initialize the solver by importing `pywraplp`.
For solving MIPs, OR-Tools utilizes a third-party open-source solver know as [SCIP (Solving Constraint Integer Programs)](https://www.scipopt.org/). We'll specify SCIP as our solver of choice in this example.

In [109]:
from ortools.linear_solver import pywraplp
solver = pywraplp.Solver.CreateSolver("SCIP")

### Parameters

#### Time

Three time steps are sufficient for demonstration purposes.

In [110]:
times = (1, 2, 3)

#### Costs and prices

Our power plant has fixed fuel costs and start-up costs. Meanwhile, the market price changes throughout the optimization horizon.


Time step | Variable costs ($C^F$) | Start-up costs ($C^{ST}$) | Market price ($\pi_t$)
---|---|---|---
1 | 120 | 10 | 200
2 | 120 | 10 | 100
3 | 120 | 10 | 200

#### Production limits

Our power plant can produce between 10 ($P^{\min}$) and 100 MW ($P^{\max}$). If it was between 0 and 100 MW, we could have modelled this as a linear program (LP).

These are essentially all the input parameters we need:

In [111]:
PROD_MIN = 10  # in MW
PROD_MAX = 100  # in MW
PROD_COSTS = {t: 120 for t in times}  # in EUR/MWh
START_UP_COSTS = 10  # in EUR
PRICE = {1: 200, 2: 100, 3: 200}  # in EUR/MWh

### Variables

In total, four variables are needed. Only one that reflects the dispatch volume and three to model the unit-commitment logic, which we will dive into when adding the constraints to the model.


- $p_t \in [P^{\min}, P^{\max}]$: production
- $u_t \in \{0,1\}$: in operation
- $v_t \in \{0,1\}$: start-up
- $w_t \in \{0,1\}$: shut-down

In the code, we store the variables that we add with either `IntVar` or `NumVar` into dictionaries so that we can access them later to write the constraints.

In [112]:
production = dict.fromkeys(times)
operating = dict.fromkeys(times)
start_up = dict.fromkeys(times)
shut_down = dict.fromkeys(times)


for t in times:
    operating[t] = solver.IntVar(0, 1, f"operating_{t}")
    start_up[t] = solver.IntVar(0, 1, f"start_up_{t}")
    shut_down[t] = solver.IntVar(0, 1, f"shut_down_{t}")
    production[t] = solver.NumVar(0, PROD_MAX, f"production_{t}")

### Objective function

Our objective is to maximize profits. In every time step, our revenue is defined as the market price minus our fuel costs for the energy produced: $(\pi_t - C^F) \cdot p_t$. In addition, the start-up costs $C^{ST}$ need to be added for start-ups $v_t$. Costs for shutting down the power plant are neglected. This yields the following objective function:

$$\max_{\mathbf{v}, \mathbf{p}}  \sum_{t \in T} (\pi_t - C^F) \cdot p_t + C^{ST} \cdot v_t.$$

We need to do the same in the code. By making use of a list comprehension, we can add it via `solver.Sum()` to the solver.

In [113]:
obj_expr = [
    (PRICE[t] - PROD_COSTS[t]) * production[t]
    - START_UP_COSTS * start_up[t]
    for t in times
]
solver.Maximize(solver.Sum(obj_expr))

### Constraints

#### Production limits

We need to add constraints for the production limits. Given that $P^{\min} > 0$, it is necessary to add the commitment variable $u_t$. Otherwise the production $p_t$ could never be zero when the power plant is switched off.

$$ u_t \cdot P^{\min} \leq p_t \leq u_t \cdot P^{\max}$$

In the code, it could make sense to split this constraint up into two constraints for the sake of clarity. In Or-Tools, constraints are added with `solver.Add()`. This is required for each time step and that is why we need a for loop here.

In [114]:
for t in times:
    solver.Add(operating[t] * PROD_MIN <= production[t])
    solver.Add(production[t] <= operating[t] * PROD_MAX)

#### Unit-commitment logic

There is at least one constraint needed to link the unit-commitment integer variables together. The chosen formulation is as follows:

$$u_t = u_{t-1} + v_t - w_t.$$

This equation ensures that if a power plant starts at time $t$ (indicated by a positive $v_t$), it is considered operational in that period due to $u_t = \dots + v_t \dots$. There are also other formulations out there in which you might need to start the power plant in $t-1$ so that is operational in $t$. This can be useful when modelling more complex start-up constraints. However, I find the formulation I use here more intuitive and compact.

In the code, we need to start iterating over time steps from the second interval, as the initial step lacks a preceding time step ($t-1$). Consequently, to manage the commitment status in the first time step, an additional constraint is needed. This constraint ensures the power plant is marked as operational (committed) if it is started at $t=1$, addressing the absence of a prior state.


In [115]:
for t in times[1:]:
    solver.Add(operating[t] == operating[t - 1] + start_up[t] - shut_down[t])

solver.Add(operating[1] == start_up[1])

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7f5fd1387840> >

## Solving

Let's solve this profit maximization problem and catch the solution (`status`).

In [116]:
status = solver.Solve()

Let's first check whether the optimization was solved successfully and what the optimal objective value is.

In [117]:
if status == pywraplp.Solver.OPTIMAL:
    print("Objective value =", solver.Objective().Value())


Objective value = 15980.0


To get to this solution, the solver needed:

In [118]:
print(f"{solver.wall_time():d} milliseconds.")

109 milliseconds.


The solver needed to explore:

In [119]:
print(f"{solver.nodes():d} branch-and-bound nodes.")

0 branch-and-bound nodes.


That is because the MIP and LP results are exactly the same. Hence, there is no need to further add branches.

Let's inspect the optimal dispatch.

In [120]:
for t in times:
    print(production[t].solution_value())

100.0
0.0
100.0


The power plants is operating with the maximum admissible production capacity when the market price is above the variable and start-up costs. This implies that the power plant is switched off during the second time interval, which we can confirm when inspecting the integer variables.

In [121]:
print("Operation ------------------------------")
for t in times:
    print(operating[t].solution_value())

print("start-up ------------------------------")
for t in times:
    print(start_up[t].solution_value())

print("shut-down ------------------------------")
for t in times:
    print(shut_down[t].solution_value())

Operation ------------------------------
1.0
0.0
1.0
start-up ------------------------------
1.0
0.0
1.0
shut-down ------------------------------
-0.0
1.0
0.0


## What's next

If you wanted to extend the scope of this unit-commitment and dispatch optimization, the next steps would include adding ramping constraints as well as the minimum uptime and downtime. These kind of constraints could be simply added as a combination of for loops and `solver.Add()`. In that sense, OR-Tools could get you quite far with a, in my opinion, nice and clean syntax.

## Summary
OR-Tools, supported by Google, is a comprehensive solution for tackling a wide range of optimization problems, including linear, integer, and constraint programming. Our exploration included a hands-on example with a unit-commitment and dispatch model, showcasing the clean and modern syntax that makes OR-Tools appealing for project development. However, the API reference could be more user-friendly. Despite that, I could see myself using OR-Tools for a bigger project but it is always worth checking in advance whether it really serves all your requirements.